In [ ]:
import os
from pathlib import path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
input_path = './FloodDataset/'


def print_file_sizes(input_path, subset):
	path = os.path.join(input_path, subset)
	print("Subset: {}".format(subset))
	print("-" * 40)
	if not os.path.exists(path):
		print("Path does not exist:", path)
		print()
		return

	for f in os.listdir(path):
		full = os.path.join(path, f)
		if os.path.isfile(full):
			size_mb = round(os.path.getsize(full) / 1e6, 2)
			print(f.ljust(30) + str(size_mb) + ' MB')
		elif os.path.isdir(full):
			sizes = [
				os.path.getsize(os.path.join(full, x))
				for x in os.listdir(full)
				if os.path.isfile(os.path.join(full, x))
			]
			total_mb = round(sum(sizes) / 1e6, 2)
			print(f.ljust(30) + str(total_mb) + ' MB (' + str(len(sizes)) + ' files)')
		else:
			print(f.ljust(30) + 'Unknown type')
	print()


print_file_sizes(input_path, 'train_another')
print_file_sizes(input_path, 'validation_another')
print_file_sizes(input_path, 'test_another')
print_file_sizes(input_path, 'test')



In [ ]:
import os

def delete_non_convertible_images(root_folder):
    for foldername, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:
            try:
                # Attempt to extract coordinates from the filename (without extension)
                lon, lat = map(float, os.path.splitext(filename)[0].split('_'))
            except ValueError:
                # If conversion to float fails, delete the image
                image_path = os.path.join(foldername, filename)
                print(f"Deleting invalid image: {image_path}")
                os.remove(image_path)

# Replace this path with your actual root folder
root_folder = input_path
delete_non_convertible_images(root_folder)


Traning Labels


In [ ]:
image_df = pd.DataFrame({'path': list(Path(input_path).glob(' ** /".jp*g'))})

image_df['damage' ] = image_df['path'].map(lambda| x: x.parent.stem)
mage_df['dat]_split'] = image_df['path'].map(lambda x: x.parent.parent.stem)
mage_df['location' ] = image_df['path'].map(lambda x: x.stem)
mage_df['lon'] = image_df['location' ].map(lambda x: float(x.split('_')[0]))
mage_df['lat'] = image_df['location'].map(lambda x: float(x.split('_')[-1]))
image_df['path'] = image_df['path'].map(lambda x: str(x)) # convert the path back to a string

image_df.head()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

s =3
alpha = 0.35

# get the train-validation-test splits
image_df_train = image_df[image_df['data_split' ] == 'train_another'].copy()
image_df_val = image_df[image_df['data_split' ] == 'validation_another'].copy()
image_df_test = image_df[image_df['data_split' ] == 'test_another'].copy()

# sort to ensure reproducible behaviour
image_df_train.sort_values('lat', inplace=True)
image_df_val.sort_values('lat', inplace=True)
image_df_test.sort_values('lat', inplace=True)
image_df_train.reset_index(drop=True, inplace=True)
image_df_val.reset_index(drop=True,inplace=True)
image_df_test.reset_index(drop=True,inplace=True)

ax[0].scatter(image_df_train['lon' ], image_df_train['lat'], color='C0', s=s, alpha=alpha, label='train' )
ax[0].scatter(image_df_val['lon'], image_df_val['lat'], color='C1', s=s, alpha=alpha, label='validation' )

ax[0].set_title('split', fontsize=14, fontweight='bold' )
ax[0].legend ()
ax[0].set_xlabel('longitude')
ax[0].set_ylabel('latitude')

image_df_dmg = image_df[image_df['damage' ] == 'damage'].copy()
image_df_nodmg = image_df[image_df['damage'] == 'no_damage'].copy()

image_df_dmg.reset_index(drop=True,inplace=True)
image_df_nodmg.reset_index(drop=True, inplace=True)


ax[1].scatter(image_df_dmg['lon'], image_df_dmg['lat'], color='C0', s=s, alpha=alpha, label='damage' )
ax[1].scatter(image_df_nodmg['lon'], image_df_nodmg['lat'], color='C1', s=s, alpha=alpha, label='no_damage' )

ax[1].set_title('damage status', fontsize=14, fontweight='bold' )
ax[1].legend()  
ax[1].set_xlabel('longitude')
ax[1].set_ylabel('latitude')
plt.show(fig)